## Project Scope: Building a Spam Filter using Naive Bayes

In this notebook, we will be creating a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to build a model that can accurately classify SMS messages as spam or ham (not spam) with an accuracy of at least 90%.

The original dataset has already labeled messages as spam or ham, so we will not be using any labels in our model.

### Naive Bayes Explained

The Naive Bayes algorithm is a simple yet powerful technique used primarily for classification tasks (e.g., spam detection, sentiment analysis). It belongs to a family of probabilistic classifiers that are based on applying Bayes' Theorem with a strong, or "naive," independence assumption.

1. Conditional probability
Witten as $P(A|B)$ where an event $A$ occurs given that event $B$ has occurred. For example, $P(\text{Spam}|\text{"Free"})$ is the probability that an email is spam, given that the word "Free" appears in the email.
$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$
Where $P(A \cap B)$ is the probability that both $A$ and $B$ occur.

2. Bayes' Theorem
Bayes' Theorem allows us to flip the conditional probability $P(A|B)$ to $P(B|A)$. For example:
$$P(B|A) = \frac{P(A|B)P(B)}{P(A)}$$

In the context of classification, this can be restated as $$P(Class|Features) = \frac{P(Features|Class)P(Class)}{P(Features)}$$

Where:
* $P(Class|Features)$: Posterior Probability, what we want to find, the probability that the data belongs to a certain class (what is the probability that the data is spam given certain features).
* $P(Features|Class)$: Likelihood, the probability of observing the given features, given the class is true.
* $P(Class)$: Prior Probability, the initial probability of the class before any evidence is seen. 
* $P(Features)$: Evidence, the probability of observing the features, which acts as a scaling constant. 

3. Naive Assumption
Naive Bayes uses conditional probability in the form of Bayes' Theorem to find the probability of a hypothesis (our classification) given the evidence (our data features).

The reason the algorithm is called "Naive" is due to its simplifying assumption: it assumes that all features used for classification are independent of each other, given the class.

For example, when classifying an email, it assumes that the presence of the word "coupon" is completely independent of the presence of the word "money," given the email is spam.

Mathematically, this means the likelihood term $P(\text{Features}|\text{Class})$ can be decomposed into a product of individual conditional probabilities:$$P(x_1, x_2, \dots, x_n|\text{Class}) = P(x_1|\text{Class}) \times P(x_2|\text{Class}) \times \dots \times P(x_n|\text{Class})$$

### Take a peek at the data

In [1]:
import pandas as pd
sms = pd.read_csv("SMSSpamCollection", sep='\t', header=None, names=['Label', 'SMS'])
sms

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [2]:
#check how many ham, and spam messages we have
print(sms['Label'].value_counts(normalize=True))

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64


### Training and Testing Data

We will split the dataset into 80% training and 20% testing.

In [3]:
#shuffle the dataset
shuffled = sms.sample(frac=1, random_state=42)

#split the dataset into training and testing sets
train_size = int(len(shuffled) * 0.8)
train = shuffled[:train_size].reset_index(drop=True)
test = shuffled[train_size:].reset_index(drop=True)

print("Training set size:", len(train))
print("Testing set size:", len(test))

Training set size: 4457
Testing set size: 1115


we would expect that in the training set and test set, the percentage of spam is about the same as in the full dataset(13%). 

In [4]:
print(train['Label'].value_counts(normalize = True))

Label
ham     0.866951
spam    0.133049
Name: proportion, dtype: float64


In [5]:
print(test['Label'].value_counts(normalize=True))

Label
ham     0.861883
spam    0.138117
Name: proportion, dtype: float64


### Before training: Data cleaning

As exlpained before, we would want to know the occurance of features given a certain class. $P(\text{Features}|\text{Class})$ can be decomposed into a product of individual conditional probabilities:$$P(x_1, x_2, \dots, x_n|\text{Class}) = P(x_1|\text{Class}) \times P(x_2|\text{Class}) \times \dots \times P(x_n|\text{Class})$$

Here, within an email labeled "spam", there could be words like "SECRET", "PRIZE", "CLAIM", "SECRET", "PRIZE", and "NOW". Where you could view each word as independent event that occurred given the class "spam".

So, what we want to do is to tokenize each word in the email and count the number of times it occurred given the class "spam" or "ham". 

#### Step 1: Remove punctuation and convert to lowercase in training data

In [6]:
import string
punctuation = '[' +string.punctuation + ']'
train['SMS'] = train['SMS'].str.replace(punctuation, ' ', regex=True)
train['SMS'] = train['SMS'].str.lower()
train.head()


,Label,SMS
0,ham,squeeeeeze this is christmas hug if u lik ...
1,ham,and also i ve sorta blown him off a couple tim...
2,ham,mmm thats better now i got a roast down me i...
3,ham,mm have some kanji dont eat anything heavy ok
4,ham,so there s a ring that comes with the guys cos...


#### Step 2: Tokenizing the vocabulary

In [7]:
train['SMS'] = train['SMS'].str.split()
vocab = []
for i in train['SMS']:
    for j in i:
        vocab.append(j)
vocab = set(vocab) #reduce replicates
vocab = list(vocab)


#### Step 3: Create a new dataframe for training that counts the number of words in each message

In [9]:
train['SMS']

0       [squeeeeeze, this, is, christmas, hug, if, u, ...
1       [and, also, i, ve, sorta, blown, him, off, a, ...
2       [mmm, thats, better, now, i, got, a, roast, do...
3       [mm, have, some, kanji, dont, eat, anything, h...
4       [so, there, s, a, ring, that, comes, with, the...
                              ...                        
4452    [solve, d, case, a, man, was, found, murdered,...
4453    [maybe, if, you, woke, up, before, fucking, 3,...
4454    [me, hungry, buy, some, food, good, lei, but, ...
4455         [wow, v, v, impressed, have, funs, shopping]
4456    [yo, you, around, a, friend, of, mine, s, look...
Name: SMS, Length: 4457, dtype: object

In [10]:
vocab

['frosty',
 'semester',
 'today',
 'silently',
 '1pm',
 '5free',
 'rightly',
 'nmde',
 'jade',
 'take',
 'independence',
 'biatch',
 '6hrs',
 'champneys',
 'cuppa',
 'charts',
 'send',
 'ldnw15h',
 'racal',
 'effects',
 'uawake',
 'aging',
 'chief',
 'staying',
 'defeat',
 'online',
 'throws',
 'guild',
 'alcohol',
 'ls1',
 'buzzzz',
 '9153',
 '078',
 'rajini',
 'urgoin',
 'tata',
 'treadmill',
 'not',
 'company',
 'meets',
 'ajith',
 'sozi',
 'aunty',
 'choose',
 '4wrd',
 'arguments',
 'playing',
 'beach',
 'terry',
 'divorce',
 '7250',
 'anand',
 '11mths',
 'cute',
 'sura',
 'vomit',
 'page',
 'pay',
 'disclose',
 'overa',
 'ultimately',
 'eight',
 'browse',
 'kb',
 '40mph',
 'juliana',
 'gap',
 'awake',
 '£71',
 'toothpaste',
 'constantly',
 'pod',
 'onto',
 'kr',
 'amp',
 'thesedays',
 'london',
 'thanx4',
 'pansy',
 'chickened',
 'j',
 'paragon',
 'magic',
 'shining',
 'paranoid',
 'ruin',
 '09058094455',
 'yeesh',
 'pharmacy',
 'mquiz',
 '09053750005',
 'sooooo',
 'csbcm4235wc1n3

In [11]:
#create a dictionary that holds word and their frequency
word_count_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocab}

#add up the number of times each word appears in each sms
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_count_per_sms[word][index] += 1

word_counts = pd.DataFrame(word_count_per_sms)
word_counts.head()

,frosty,semester,today,silently,1pm,5free,rightly,nmde,jade,take,...,person,elections,where,question,paul,uptown,matthew,banned,respect,college
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#combine original train and word_counts dataframes
train_cleaned = pd.concat([train, word_counts], axis=1)
train_cleaned.head()

,Label,SMS,frosty,semester,today,silently,1pm,5free,rightly,nmde,...,person,elections,where,question,paul,uptown,matthew,banned,respect,college
0,ham,"[squeeeeeze, this, is, christmas, hug, if, u, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[and, also, i, ve, sorta, blown, him, off, a, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[mmm, thats, better, now, i, got, a, roast, do...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[mm, have, some, kanji, dont, eat, anything, h...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[so, there, s, a, ring, that, comes, with, the...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Spam Filter Creation

Now we have a training set that contains the frequency of each word in the spam and ham messages, we also need 4 key probabilities to make predictions:
1. P(Spam)
2. P(Ham)
3. P(word | Spam)
4. P(word | Ham)

The 2 main questions we need to answer are:
1. What is the probability of a message being spam given the words in the message $P(Spam | word)$?
2. What is the probability of a message being ham given the words in the message $P(Ham | word)$?

Eventually, the filter that we build will predict the class (spam or ham) that maximizes this probability $P(Class|word)$.

#### Estimation formula - Pre calculation

Here we estimate the probabilities from our observed data (the training set). The probability of an event is estimated by its Relative Frequency: $$P(\text{Event}) \approx \frac{\text{Count of Event Occurrences}}{\text{Total Number of Trials}}$$


*Note: In order to prevent 0 probabilities, we will use the Laplace smoothing technique -> alpha = 1*

$$\mathbf{P(w_i|\text{Spam})} = \frac{N_{w_i|\text{Spam}} + \alpha}{N_{\text{Spam}} + \alpha \cdot N_{\text{Vocabulary}}}$$

$$\mathbf{P(w_i|\text{Ham})} = \frac{N_{w_i|\text{Ham}} + \alpha}{N_{\text{Ham}} + \alpha \cdot N_{\text{Vocabulary}}}$$

In detail:
* Denominator: Represents the total count of words available within the Spam class, adjusted for smoothing -> (Total Spam word count) + (alpha * Total Vocabulary Size)
* Numerator: (Count of how many times the specific word $w_i$ appeared in all the Spam messages in the training data combined) + (alpha)

In [ ]:
alpha = 1

is_spam = train_cleaned['Label'] == 'spam'
is_ham = train_cleaned['Label'] == 'ham'
spam = train_cleaned[is_spam]
ham = train_cleaned[is_ham]
spam_indices = spam.index.tolist()
ham_indices = ham.index.tolist()

#P(Spam) & P(Ham)
p_spam = len(spam) / len(train_cleaned)
p_ham = len(ham) / len(train_cleaned)

#N_Spam & N_Ham -> Number of words in spam and ham
n_spam = spam['SMS'].apply(len).sum()
n_ham = ham['SMS'].apply(len).sum()

#N_vocabulary -> Number of unique words in the vocabulary
n_vocab = len(vocab) #vocab is already unique

#derive N_w_spam & N_w_ham first, then P(w|spam) & P(w|ham)
parameter_spam = {unique_word: 0 for unique_word in vocab}
parameter_ham = {unique_word: 0 for unique_word in vocab}
word_counts_per_sms = {unique_word: [0] * len(train_cleaned) for unique_word in vocab}

for index, sms in enumerate(train_cleaned['SMS']):
    for word in sms:
        if word in word_counts_per_sms: # Check if the word is in the vocabulary
             word_counts_per_sms[word][index] += 1

for word in vocab:
    # Get the list of counts for the current word across ALL messages
    all_counts = word_counts_per_sms[word]
    
    # Calculate N_wi|Spam: Sum the counts only at the indices that correspond to spam messages
    N_wi_spam = sum(all_counts[i] for i in spam_indices)
    
    # Calculate N_wi|Ham: Sum the counts only at the indices that correspond to ham messages
    N_wi_ham = sum(all_counts[i] for i in ham_indices)
    
    # Apply the formula (Likelihood P(word|Spam) = (Count + alpha) / (Total_Spam_Words + alpha * N_Vocab))
    parameter_spam[word] = (N_wi_spam + alpha) / (n_spam + alpha * n_vocab)
    parameter_ham[word] = (N_wi_ham + alpha) / (n_ham + alpha * n_vocab)

#### Putting it all together - Classifying a new message

From the calculation above, we have get 4 parameters:
1. P(Spam)
2. P(Ham)
3. P(word | Spam)
4. P(word | Ham)

Now we can use these parameters to classify new messages. The spam filter can be implemented as follows:
* Takes a message (w1, w2, w3, ...) as input
* Calculate P(Spam | w1, w2, w3, ...) and P(Ham | w1, w2, w3, ...)
* If P(Spam | w1, w2, w3, ...) > P(Ham | w1, w2, w3, ...), classify as Spam
* Otherwise, classify as Ham

We have derived from the above expression that there is a way to flip from P(word|Class) to P(Class|word):
$$P(\text{Class}|\text{word}) = \frac{P(\text{word} \cap \text{Class})}{P(\text{Class})} \times \frac{P(\text{Class})}{P(\text{word})}$$

Essentially, we can derive the P(word|Class) first, then do multiplication.



In [27]:
#P(Spam) = p_spam
#P(Ham) = p_ham
#P(word|Spam) = parameter_spam
#P(word|Ham) = parameter_ham
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
            
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
        return 'spam'
    else:
        print('Equal proabilities, have a human classify this!')
        return 'uncertain'


<>:12: SyntaxWarning: invalid escape sequence '\W'
<>:12: SyntaxWarning: invalid escape sequence '\W'
C:\Users\USER\AppData\Local\Temp\ipykernel_21024\2854449576.py:12: SyntaxWarning: invalid escape sequence '\W'
  message = re.sub('\W', ' ', message)


The goal of the loop is to calculate the Posterior Probability for the Spam class using the following proportional relationship derived from Bayes' Theorem:$$P(\text{Spam}|\text{message}) \propto P(\text{Spam}) \times P(w_1|\text{Spam}) \times P(w_2|\text{Spam}) \times \dots \times P(w_n|\text{Spam})$$

In [17]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.4913314003869138e-25
P(Ham|message): 1.2147013987554955e-27
Label: Spam


In [18]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.639214349619395e-25
P(Ham|message): 2.7792500399144073e-21
Label: Ham


### Test it out

In [28]:
test['predicted'] = test['SMS'].apply(classify)
test.head()

P(Spam|message): 9.165881359117024e-22
P(Ham|message): 2.108367576160899e-19
Label: Ham
P(Spam|message): 3.337138093453127e-33
P(Ham|message): 1.2132671970118582e-27
Label: Ham
P(Spam|message): 2.350587218410885e-32
P(Ham|message): 1.940826929051198e-27
Label: Ham
P(Spam|message): 5.528304952332478e-25
P(Ham|message): 2.4649867199677225e-21
Label: Ham
P(Spam|message): 3.5001913498194867e-35
P(Ham|message): 6.6218099714613355e-28
Label: Ham
P(Spam|message): 1.3570410580595237e-95
P(Ham|message): 1.3191531319952736e-73
Label: Ham
P(Spam|message): 7.962974877693402e-59
P(Ham|message): 3.491170313426637e-46
Label: Ham
P(Spam|message): 4.588150996615058e-18
P(Ham|message): 8.806515056476192e-13
Label: Ham
P(Spam|message): 1.5763470458213e-20
P(Ham|message): 3.0561560345075486e-16
Label: Ham
P(Spam|message): 7.859386046965392e-26
P(Ham|message): 1.634646487052676e-18
Label: Ham
P(Spam|message): 4.251509410175817e-71
P(Ham|message): 7.582038017040817e-88
Label: Spam
P(Spam|message): 4.5881509

,Label,SMS,predicted
0,ham,I fetch yun or u fetch?,ham
1,ham,Was playng 9 doors game and gt racing on phone...,ham
2,ham,I dont thnk its a wrong calling between us,ham
3,ham,All e best 4 ur exam later.,ham
4,ham,Hey what how about your project. Started aha da.,ham


In [29]:
#Accuracy 
correct = 0
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
print(correct / test.shape[0])

0.979372197309417
